In [103]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact

In [104]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [105]:
#Choose Patient
study='BIGAZZI'

In [109]:
#rp=dcm.read_file('/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/dicom/RP.201700810.sternum.dcm')
rp2=dcm.read_file('/media/sf_Documents/Thesis/Work/DATA/Clinic/CasClinique/electron/BAKER/dicom/RP.201610113.AILEDENEZD_TT.dcm')
#rd=dcm.read_file(study+'/dicom/rd_joue.dcm')
print(rp2)
#get pixel

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20180726'
(0008, 0013) Instance Creation Time              TM: '161835.270000'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.5.950586737223.451322.20170124173856
(0008, 0020) Study Date                          DA: '20170111'
(0008, 0030) Study Time                          TM: '114828'
(0008, 0050) Accession Number                    SH: '103093944'
(0008, 0060) Modality                            CS: 'RTPLAN'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: 'CHIRA^CIPRIAN'
(0008, 1010) Station Name                        SH: 'SRVVARIAN-DB'
(0008, 1030) Study Description                   LO: 'TDM therapie SPC'
(0008, 103e) Series Description                  LO: 'Plans ARIA RadOnc'
(0008, 1048) P

### Edit rt dose.dcm in raw file

In [92]:
rp=dcm.read_file('/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/dicom/RP.201700806.LIT Tumoral.dcm')
rd=dcm.read_file('/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/dicom/RD.201700806.Dose_LIT Tumoral_LIT Tumoral.dcm')

#get pixel array
rdpix=rd.pixel_array.astype(np.float32)


#Convert rt dose value into Gy value 
f=float(rd.DoseGridScaling)
print(f)
rdpix1=rdpix*f 

#create an sitk image from this array
rdim=sitk.GetImageFromArray(rdpix1)

#Set origin of image from rt dose value
origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)
#Set spacing of image from rt dose value (x et y)
spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))
#for z direction
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)

#write rd image in '.raw'
sitk.WriteImage(rdim, '/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/results/eclipse_'+study+'.mhd')
print('eclipse_'+study+'.mhd --> ok')

1.3721952e-06
eclipse_BIGAZZI.mhd --> ok


In [93]:
#Get the rd raw created  and gate image
refim=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/results/eclipse_'+study+'.mhd')
gateim=sitk.ReadImage('/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/results/bigazzi_out.mhd')

In [94]:
# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)                                          
ct_viewer(refpix)
ct_viewer(gatepix)

interactive(children=(IntSlider(value=72, description=u'myslice', max=145), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=36, description=u'myslice', max=73), Output()), _dom_classes=('widget-interact',))

In [96]:
#to mask GATE images an get just BODY DOSE
#Resample (Downsampling) rd image into gate image size an resolution
refim_res=sitk.Resample(refim, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
#Get pixel array
refim_respix=sitk.GetArrayFromImage(refim_res)
ct_viewer(refim_respix)

interactive(children=(IntSlider(value=36, description=u'myslice', max=73), Output()), _dom_classes=('widget-interact',))

In [97]:
#Write resampled image
sitk.WriteImage(refim_res, '/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/results/eclipse_'+study+'_resampled.mhd')
print('eclipse_'+study+'_resampled.mhd --> ok')

eclipse_BIGAZZI_resampled.mhd --> ok


In [98]:
#Create a boolean mask to apply on Gate image (remove dose outside the patient according to RTdose)
mask=np.ma.make_mask(refim_respix)
#apply on gate pixel array
gatepix_mask=gatepix*mask
ct_viewer(gatepix_mask)

interactive(children=(IntSlider(value=36, description=u'myslice', max=73), Output()), _dom_classes=('widget-interact',))

In [99]:
energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
if energy == 6:
    GVref=0.000108726
    Npart=1033128667.0
    DrG=0.000108726
    
if energy == 9:
    GVref=0.000140945
    Npart=1136273663.0
    DrG=0.000140945

if energy == 12:
    GVref=0.000150559
    Npart=1086793583.0
    DrG=0.000150559

In [100]:
energy=rp.BeamSequence[0].ControlPointSequence[0].NominalBeamEnergy
print(energy)
#Absolute dose conversion LUC SIMON
um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

#GVref6=0.000108726
#GVref9=0.000140945
#GVref12=0.000150559

gatepix_abs=gatepix_mask*(1.0/GVref)*0.01*um*nFrac
print(gatepix_abs.max())

9
11.5205


In [101]:
#Absolute dose conversion BIBLIO

#Npart_6=1033128667.0
#Npart_9=1136273663.0
#Npart_12=1086793583.0
#DrG_6=0.000108726
#DrG_9=0.000140945
#DrG_12=0.000150559

GperPart=DrG/Npart

Dref=0.01

F=Dref/GperPart

um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

gatepix_abs=(gatepix_mask/Npart)*F*um*nFrac
#print(gatepix_mask.max())
print(gatepix_abs.max())


11.5205


In [102]:

#Create a raw image of gate with mask
#gate_dose=sitk.GetImageFromArray(gatepix_mask)
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(gate_dose, '/media/sf_CRCT/JupyterNotebook/data/image/ct/'+study+'/results/gate_'+study+'.mhd')